<a href="https://colab.research.google.com/github/AntonioLunardi/Challenge-Data-Science-Alura-2ed/blob/main/Challenge_DS_Alura_2_Ed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.0 Inicialização do PySpark e importação do dataset

In [26]:
!pip install pyspark==3.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [28]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz

In [29]:
!tar xf spark-3.3.1-bin-hadoop3.tgz

In [30]:
!pip install -q findspark

In [31]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [32]:
import findspark
findspark.init()

In [33]:
from pyspark.sql import SparkSession

In [34]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Iniciando com Spark') \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [35]:
spark

In [36]:
import json

dataset_bruto = spark.read.option('multiline', 'true').json('/content/drive/MyDrive/Alura/Challenge_DS_2ed/dataset_bruto.json')

# 1.0 Engenharia de Dados

## 1.1 Visualização inicial

[Base de dados - InsightPlaces](https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip)

### Dicionário de Dados - Anuncio

| Colunas         | Descrição                                                      |
|-----------------|----------------------------------------------------------------|
| id              | Código de identificação do anúncio no sistema da InsightPlaces |
| tipo_unidade    | Tipo de imóvel (apartamento, casa e outros)                    |
| tipo_uso        | Tipo de uso do imóvel (residencial ou comercial)               |
| area_total      | Área total do imóvel (construção e terreno)                    |
| area_util       | Área construída do imóvel                                      |
| quartos         | Quantidade de quartos do imóvel                                |
| suites          | Quantidade de suítes do imóvel                                 |
| banheiros       | Quantidade de banheiros do imóvel                              |
| vaga            | Quantidade de vagas de garagem do imóvel                       |
| caracteristicas | Listagem de características do imóvel                          |
| andar           | Número do andar do imóvel                                      |
| endereco        | Informações sobre o endereço do imóvel                         |
| valores         | Informações sobre valores de venda e locação dos imóveis       |

In [37]:
dataset_bruto.printSchema() # Esquema de registro no arquivo JSON

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

In [38]:
quantidade_registros = dataset_bruto.count() # Quantidade de registros
quantidade_entidades = len(dataset_bruto.columns) # Entidades computadas (structs e arrays)

print('Quantidade de registros:', quantidade_registros)
print('\nQuantidade de entidades:', quantidade_entidades)

Quantidade de registros: 89083

Quantidade de entidades: 3


## 1.2 Seleção de features e transformação em colunas

In [39]:
# Transformando os dicionários JSON em colunas e selecionando features:
# A pedido do time de ciência de dados, não serão incluídas as colunas de endereço além de bairro e zona

dataset_anuncio_com_arrays = dataset_bruto\
  .select('anuncio.andar', 'anuncio.area_total', 'anuncio.area_util', 'anuncio.banheiros', 'anuncio.caracteristicas',\
          'anuncio.endereco.bairro', 'anuncio.endereco.zona', 'anuncio.id','anuncio.quartos', 'anuncio.suites',\
          'anuncio.tipo_anuncio', 'anuncio.tipo_unidade', 'anuncio.tipo_uso', 'anuncio.vaga',\
          'anuncio.valores.condominio', 'anuncio.valores.iptu', 'anuncio.valores.tipo', 'anuncio.valores.valor')

dataset_anuncio_com_arrays.show()

+-----+----------+---------+---------+--------------------+--------------+------------+--------------------+-------+------+------------+------------+-----------+----+----------+------+-------+-------+
|andar|area_total|area_util|banheiros|     caracteristicas|        bairro|        zona|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|condominio|  iptu|   tipo|  valor|
+-----+----------+---------+---------+--------------------+--------------+------------+--------------------+-------+------+------------+------------+-----------+----+----------+------+-------+-------+
|    0|        []|     [16]|      [0]|                  []|        Centro|Zona Central|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|  Comercial| [1]|     [260]| [107]|[Venda]|[10000]|
|    0|        []|     [14]|      [0]|                  []|        Centro|Zona Central|b6ffbae1-17f6-487...|    [0]|    []|       Usado|      Outros|  Comercial| [0]|     [260]| [107]|[Venda]|[100

In [40]:
# Tranformando as colunas de arrays em strings para facilitar as análises e tratamentos

from pyspark.sql.functions import concat_ws

dataset_anuncio = dataset_anuncio_com_arrays # Trocando a váriável trabalhada

dataset_anuncio = dataset_anuncio.withColumn('area_total', concat_ws(',', 'area_total'))
dataset_anuncio = dataset_anuncio.withColumn('area_util', concat_ws(',', 'area_util'))
dataset_anuncio = dataset_anuncio.withColumn('banheiros', concat_ws(',', 'banheiros'))
dataset_anuncio = dataset_anuncio.withColumn('caracteristicas', concat_ws(',', 'caracteristicas'))
dataset_anuncio = dataset_anuncio.withColumn('quartos', concat_ws(',', 'quartos'))
dataset_anuncio = dataset_anuncio.withColumn('suites', concat_ws(',', 'suites'))
dataset_anuncio = dataset_anuncio.withColumn('vaga', concat_ws(',', 'vaga'))
dataset_anuncio = dataset_anuncio.withColumn('condominio', concat_ws(',', 'condominio'))
dataset_anuncio = dataset_anuncio.withColumn('iptu', concat_ws(',', 'iptu'))
dataset_anuncio = dataset_anuncio.withColumn('tipo', concat_ws(',', 'tipo'))
dataset_anuncio = dataset_anuncio.withColumn('valor', concat_ws(',', 'valor'))

dataset_anuncio.show() # Dataset após as modificações

+-----+----------+---------+---------+--------------------+--------------+------------+--------------------+-------+------+------------+------------+-----------+----+----------+----+-----+-----+
|andar|area_total|area_util|banheiros|     caracteristicas|        bairro|        zona|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|condominio|iptu| tipo|valor|
+-----+----------+---------+---------+--------------------+--------------+------------+--------------------+-------+------+------------+------------+-----------+----+----------+----+-----+-----+
|    0|          |       16|        0|                    |        Centro|Zona Central|47d553e0-79f2-4a4...|      0|     0|       Usado|      Outros|  Comercial|   1|       260| 107|Venda|10000|
|    0|          |       14|        0|                    |        Centro|Zona Central|b6ffbae1-17f6-487...|      0|      |       Usado|      Outros|  Comercial|   0|       260| 107|Venda|10000|
|    0|      1026|     10

## 1.3 Seleção dos valores de venda

In [41]:
# Verificando todas as categorias de registros de tipo (4 tipos).
# O anunciante pode registrar valores de venda ou de alguel na ordem que prefira.
# Assim, os valores registrados nos arrays 'valor', 'iptu' e 'condomínio' seguem a referência estabelecida por 'tipo'

dataset_anuncio_com_arrays\
    .groupby('tipo')\
    .count()\
    .show()

+----------------+-----+
|            tipo|count|
+----------------+-----+
|[Venda, Aluguel]| 1028|
|  [Venda, Venda]|   92|
|[Aluguel, Venda]|  811|
|         [Venda]|87152|
+----------------+-----+



In [42]:
# Visualizando como ficaram os 5 linhas de valores de um dos quatro tipos:

# 'Aluguel,Venda'
dataset_anuncio\
    .select(dataset_anuncio.valor.alias('Valor: Aluguel, Venda')\
            ,dataset_anuncio.iptu\
            ,dataset_anuncio.condominio)\
    .where(dataset_anuncio.tipo=='Aluguel,Venda')\
    .show(5)

# 'Venda'
dataset_anuncio\
    .select(dataset_anuncio.valor.alias('Valor: Venda')\
           ,dataset_anuncio.iptu, dataset_anuncio.condominio)\
    .where(dataset_anuncio.tipo=='Venda')\
    .show(5)

# 'Venda,Venda'
dataset_anuncio\
    .select(dataset_anuncio.valor.alias('Valor: Venda, Venda')\
            ,dataset_anuncio.iptu, dataset_anuncio.condominio)\
    .where(dataset_anuncio.tipo=='Venda,Venda')\
    .show(5)

# 'Venda,Aluguel'
dataset_anuncio\
    .select(dataset_anuncio.valor.alias('Valor: Venda, Aluguel')\
            ,dataset_anuncio.iptu, dataset_anuncio.condominio)\
    .where(dataset_anuncio.tipo=='Venda,Aluguel')\
    .show(5)

+---------------------+-------+----------+
|Valor: Aluguel, Venda|   iptu|condominio|
+---------------------+-------+----------+
|            500,15000|    117|       500|
|            250,38000|107,107|   295,295|
|            100,44000|126,126|   600,600|
|            550,60000|115,115|       0,0|
|            800,55000|    0,0|   200,200|
+---------------------+-------+----------+
only showing top 5 rows

+------------+----+----------+
|Valor: Venda|iptu|condominio|
+------------+----+----------+
|       10000| 107|       260|
|       10000| 107|       260|
|       10000|1613|          |
|       10000|    |        80|
|        5000|   0|         0|
+------------+----+----------+
only showing top 5 rows

+-------------------+----+----------+
|Valor: Venda, Venda|iptu|condominio|
+-------------------+----+----------+
|     149000,2263000| 0,0|       0,0|
|      159000,169000| 0,0|       0,0|
|      175000,300864| 0,0|       0,0|
|      189000,239000| 0,0|       0,0|
|      189000,2990

In [ ]:
# A equipe de ciência de dados solicitou que fossem transmitidos somente os valores de venda (valor, iptu e condomínio)
# para posterior análise de machine learning.

# Os valores sobrescritos foram os posteriores à vírgula apenas para o tipo 'Aluguel,Venda', pois é o único caso
# em que o valor pertinente está na segunda posição. Nos demais, o valor da string foi subtituído pelo valor 
# anterior à vírgula para os três valores de venda (valor, iptu, condomínio).

from pyspark.sql.functions import substring_index, when

# Substituição de 'valor'
dataset_anuncio = dataset_anuncio\
    .withColumn('valor'\
                ,when(dataset_anuncio.tipo != ('Aluguel,Venda')\
                      ,substring_index(dataset_anuncio.valor, ',', 1))\
                .otherwise(substring_index(dataset_anuncio.valor, ',', -1)))

# Substituição de 'iptu'
dataset_anuncio = dataset_anuncio\
    .withColumn('iptu'\
                ,when(dataset_anuncio.tipo != ('Aluguel,Venda')\
                      ,substring_index(dataset_anuncio.iptu, ',', 1))\
                .otherwise(substring_index(dataset_anuncio.iptu, ',', -1)))
    
# Substituição de 'iptu'
dataset_anuncio = dataset_anuncio\
    .withColumn('condominio'\
                ,when(dataset_anuncio.tipo != ('Aluguel,Venda')\
                      ,substring_index(dataset_anuncio.condominio, ',', 1))\
                .otherwise(substring_index(dataset_anuncio.condominio, ',', -1)))
    
# São selecionados novamente os valores das categorias, desta vez apenas com o componente de venda

dataset_anuncio\
    .select(dataset_anuncio.valor.alias('Valor: Aluguel, Venda')\
            ,dataset_anuncio.iptu\
            ,dataset_anuncio.condominio)\
    .where(dataset_anuncio.tipo=='Aluguel,Venda')\
    .show(5)

dataset_anuncio\
    .select(dataset_anuncio.valor.alias('Valor: Venda')\
           ,dataset_anuncio.iptu, dataset_anuncio.condominio)\
    .where(dataset_anuncio.tipo=='Venda')\
    .show(5)

dataset_anuncio\
    .select(dataset_anuncio.valor.alias('Valor: Venda, Venda')\
            ,dataset_anuncio.iptu, dataset_anuncio.condominio)\
    .where(dataset_anuncio.tipo=='Venda,Venda')\
    .show(5)

dataset_anuncio\
    .select(dataset_anuncio.valor.alias('Valor: Venda, Aluguel')\
            ,dataset_anuncio.iptu, dataset_anuncio.condominio)\
    .where(dataset_anuncio.tipo=='Venda,Aluguel')\
    .show(5)

+---------------------+----+----------+
|Valor: Aluguel, Venda|iptu|condominio|
+---------------------+----+----------+
|                15000| 117|       500|
|                38000| 107|       295|
|                44000| 126|       600|
|                60000| 115|         0|
|                55000|   0|       200|
+---------------------+----+----------+
only showing top 5 rows



## 1.4 Filtragem das categorias

In [ ]:
# Tabela de frequências das categorias a serem filtradas a pedido do time de ciência de dados

dataset_anuncio.groupby('tipo_uso').count().show()
dataset_anuncio.groupby('tipo_unidade').count().show()
dataset_anuncio.groupby('tipo_anuncio').count().show()

In [ ]:
# Aplicação dos filtros de categoria (residencial, apartamento e usado)

dataset_anuncio = dataset_anuncio.filter(dataset_anuncio.tipo_uso == 'Residencial')
dataset_anuncio = dataset_anuncio.filter(dataset_anuncio.tipo_unidade == 'Apartamento')
dataset_anuncio = dataset_anuncio.filter(dataset_anuncio.tipo_anuncio == 'Usado')

dataset_anuncio.groupby('tipo_uso').count().show()
dataset_anuncio.groupby('tipo_unidade').count().show()
dataset_anuncio.groupby('tipo_anuncio').count().show()

Comparando o dicionário com o df obtido do arquivo JSON, é possível verificar que há variáveis que não foram declaradas com o tipo adequado:
 - area total, area util, condomínio, iptu, valor (são strings)
 - banheiros, quartos, suites, andar, vaga (são longs)

In [ ]:
from pyspark.sql.types import DoubleType, IntegerType # Importando os tipos de variáveis

# Removendo coluna 'tipo', que não tem mais utilidade, pois só há valores de venda agora
dataset_anuncio = dataset_anuncio.drop('tipo') 

# Passando variáveis para int
dataset_anuncio = dataset_anuncio.withColumn('andar', dataset_anuncio.andar.cast(IntegerType()))
dataset_anuncio = dataset_anuncio.withColumn('area_total', dataset_anuncio.area_total.cast(IntegerType()))
dataset_anuncio = dataset_anuncio.withColumn('area_util', dataset_anuncio.area_util.cast(IntegerType()))
dataset_anuncio = dataset_anuncio.withColumn('quartos', dataset_anuncio.quartos.cast(IntegerType()))
dataset_anuncio = dataset_anuncio.withColumn('suites', dataset_anuncio.area_total.cast(IntegerType()))
dataset_anuncio = dataset_anuncio.withColumn('banheiros', dataset_anuncio.banheiros.cast(IntegerType()))

# Transformando valores em double
dataset_anuncio = dataset_anuncio.withColumn('condominio', dataset_anuncio.condominio.cast(DoubleType()))
dataset_anuncio = dataset_anuncio.withColumn('iptu', dataset_anuncio.iptu.cast(DoubleType()))
dataset_anuncio = dataset_anuncio.withColumn('valor', dataset_anuncio.valor.cast(DoubleType()))

# Esquema com as variáveis modificadas
dataset_anuncio.printSchema() 

## 1.5 Salvando como parquet

In [ ]:
dataset_anuncio.write\
    .parquet(\
             path='/content/drive/MyDrive/Alura/Challenge_DS_2ed/dataset_anuncio_tratado'\
             ,mode='overwrite')

## 1.6 Tratamento dos valores null e nan

# 2.0 Ciência de Dados - Modelo de Regressão

## 2.1 Trocando os tipos das variáveis

In [ ]:
from pyspark.sql import functions as f # Importando funções sql

In [ ]:
# Contagem dos elementos NAN
dataset_anuncio\
    .select([f.count(f.when(f.isnan(c), True)).alias(c) for c in dataset_anuncio.columns])\
    .show()

# Contagem dos elementos NULOS
dataset_anuncio\
    .select([f.count(f.when(f.isnull(c), True)).alias(c) for c in dataset_anuncio.columns])\
    .show()

# Há muitos valores que repetem o que foi registrado na coluna 'area_util', ocasionando em alta correlação.
# Optou-se por excluir a coluna 'area_total' pois ela apresenta muito mais elementos nulos que a 'area_util'.
dataset_anuncio = dataset_anuncio.drop('area_total')

# Preenchendo nulos com zeros
dataset_anuncio = dataset_anuncio.na.fill(0)

# Nova contagem dos elementos NULOS
dataset_anuncio\
    .select([f.count(f.when(f.isnull(c), True)).alias(c) for c in dataset_anuncio.columns])\

# 3.0 Ciência de Dados - Modelo de Clusterização